In [2]:
#文档向量
import numpy as np
import pandas as pd
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from gensim.parsing.preprocessing import preprocess_string,remove_stopwords
import random
import warnings
from scipy.spatial.distance import pdist
warnings.filterwarnings("ignore")

In [33]:
import scipy.io
from collections import Counter
from itertools import product
from tqdm import trange
from tqdm import tqdm
from copy import deepcopy

In [77]:
def normalize(word_vec): 
    norm=np.linalg.norm(word_vec,ord=2,axis=0) 
    if norm == 0: 
         return word_vec 
    return word_vec/norm

计算PECO的相似矩阵

In [59]:
lines = pd.read_excel(r'D:\MDA-GCNFTG-main\GDA\peco_def.xlsx')
print(lines.head())
print(type(lines))       

                          peco  \
0     sodium chloride exposure   
1             unknown exposure   
2  Pseudomonas avenae exposure   
3             greenhouse study   
4       abscisic acid exposure   

                                                 def  
0  A salt exposure (PECO:0007185) involving the u...  
1  A plant exposure (PECO:0001001) where there is...  
2  The treatment involving exposure of plant to t...  
3  The treatment where the plants were grown unde...  
4  A growth hormone exposure (PECO:0007165) invol...  
<class 'pandas.core.frame.DataFrame'>


In [3]:
class DocumentDataset(object) :
    def __init__(self,data:pd.DataFrame,column) :
        document = data[column].apply(self.preprocess)
        self.documents = [TaggedDocument(text,[index])
        for index,text in document.iteritems()]

    def preprocess(self,document) :
        return preprocess_string(remove_stopwords(document))
    
    def __iter_(self):
        for document in self.documents:
            yield documents
    def tagged_documents(self,shuffle=False) :
        if shuffle:
            random.shuffle(self.documents)
        return self.documents


In [60]:
document_dataset = DocumentDataset(lines,'def')

In [7]:
docVecModel = Doc2Vec(min_count=1,
                    window=5,
                    vector_size=100,sample=1e-4,
                    negative=5,
                    workers=2)

#TaggedDocument的实例
docVecModel.build_vocab(document_dataset.tagged_documents()) 

In [62]:
docVecModel.train(document_dataset.tagged_documents(shuffle=True),
            total_examples = docVecModel.corpus_count,
            epochs=10)

In [49]:
docVecModel.save(r'D:\MDA-GCNFTG-main\GDA\pecoVecModel.d2v')

In [83]:
pl = []
for i in range(len(lines['peco'])):    
        pl.append(docVecModel[i])
        
PSSM = np.corrcoef(pl,pl)


In [84]:
PSSM

array([[ 1.        , -0.01469787, -0.05122649, ...,  0.04645632,
         0.15341735, -0.1502182 ],
       [-0.01469787,  1.        , -0.02405414, ..., -0.09853404,
        -0.01296746,  0.02360266],
       [-0.05122649, -0.02405414,  1.        , ...,  0.17213865,
        -0.00572574, -0.11191597],
       ...,
       [ 0.04645632, -0.09853404,  0.17213865, ...,  1.        ,
        -0.00693678,  0.07415919],
       [ 0.15341735, -0.01296746, -0.00572574, ..., -0.00693678,
         1.        , -0.09784653],
       [-0.1502182 ,  0.02360266, -0.11191597, ...,  0.07415919,
        -0.09784653,  1.        ]])

In [89]:
np.savetxt(r'PSSM.txt',PSSM, fmt='%f', delimiter=',')

[[1.         0.98198051]
 [0.98198051 1.        ]]


计算gene的相似矩阵

In [4]:
lines = pd.read_excel(r'D:\MDA-GCNFTG-main\GDA\gene_name_def.xlsx')
print(lines.head())
print(len(lines))  

                                       gene  \
0                            LOC_Os01g64660   
1                            LOC_Os03g38000   
2                            LOC_Os10g20630   
3  BTH-induced ERF transcriptional factor 2   
4                    Ent-kaurene synthase 6   

                                                 def  
0  Catalysis of the reaction: D-fructose 1,6-bisp...  
1  Binding to an RNA molecule or a portion thereo...  
2                   Binding to a calcium ion (Ca2+).  
3  A transcription regulator activity that modula...  
4  Reactions, triggered in response to the presen...  
12187


In [5]:
document_dataset = DocumentDataset(lines,'def')

In [8]:
docVecModel.train(document_dataset.tagged_documents(shuffle=True),
            total_examples = docVecModel.corpus_count,
            epochs=10)

In [9]:
docVecModel.save(r'D:\MDA-GCNFTG-main\GDA\geneVecModel.d2v')

In [10]:
docVecModel[123]

array([-0.56726223,  1.0542306 , -0.01064432,  1.3304292 ,  0.57197165,
       -1.4561863 ,  0.833518  ,  0.9520376 , -1.7497698 , -0.65613073,
       -0.41294712, -0.6259643 ,  0.65125763, -0.4949115 ,  0.65485823,
        0.36468238,  0.54316765, -0.03805818, -0.19582593,  0.29681045,
       -0.15857954, -0.8587142 ,  0.08078966,  0.83996826,  0.61348504,
        0.87959194, -0.31068867,  1.5680707 ,  0.5999156 ,  1.1967789 ,
        0.9890515 ,  0.91333395, -0.27526766,  0.385907  ,  0.5644851 ,
        0.11917278,  1.5829637 ,  0.15411966,  0.36179778, -0.5463829 ,
       -1.0048064 ,  1.4320352 , -0.17996225,  0.21984331, -0.6032092 ,
        0.06404587,  0.89795256,  0.57576364,  1.5562563 , -0.5847714 ,
       -0.47722027, -0.2865732 ,  0.5499055 ,  0.58250254,  0.620217  ,
        0.8915011 , -0.96515924, -0.21736196,  0.01693996,  0.7842304 ,
       -0.422251  ,  1.9963826 ,  0.8494391 ,  0.3830772 , -1.557347  ,
        0.8469537 , -1.0322317 , -0.77306306,  0.87841314, -1.02

In [11]:
gl = []
for i in range(len(lines['gene'])):    
    gl.append(docVecModel[i])
        
GSSM = np.corrcoef(gl,gl)

In [20]:
np.savetxt('GSSM.txt', GSSM, fmt='%.3f')

In [25]:
peco_name = pd.read_excel('.\data\peco_name.xlsx', header = None, names = ['peco'])
peco_name['peco_idx'] = peco_name.index

gene_name = pd.read_excel('.\data\gene_name.xlsx', header = None, names = ['gene'])
gene_name['gene_idx'] = gene_name.index

known_associations = pd.read_excel('.\data\known_associations.xlsx', header = None, names = ['peco','gene'])
known_associations = pd.merge(known_associations, peco_name, on = 'peco')
known_associations =  pd.merge(known_associations, gene_name, on = 'gene')
known_associations['label'] = 1
known_associations = known_associations.drop_duplicates().reset_index(drop = True)
known_associations

,peco,gene,peco_idx,gene_idx,label
0,unknown exposure,13113.t00029,1,104,1
1,Magnaporthe grisea exposure,13113.t00029,12,104,1
2,unknown exposure,13113.t00060,1,11064,1
3,continuous dark (no light) exposure,13113.t00060,20,11064,1
4,unknown exposure,13113.t00104,1,10233,1
...,...,...,...,...,...
23553,benzothiadiazole exposure,LOC_Os10g40950,27,2372,1
23554,methyl jasmonate exposure,LOC_Os02g55990,14,4481,1
23555,methyl jasmonate exposure,LOC_Os04g44670,14,3338,1
23556,methyl jasmonate exposure,LOC_Os10g10030,14,4812,1


In [30]:
print(list(set(peco_name['peco_idx'])))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [31]:
positive_gep = known_associations[['peco_idx', 'gene_idx']].sort_values(by = ['peco_idx', 'gene_idx'], ascending = True).reset_index(drop = True)
len(set(positive_gep['peco_idx'])), len(set(positive_gep['gene_idx']))

(24, 11177)

In [34]:
peco_idx = list(set(positive_gep['peco_idx']))
gene_idx = list(set(positive_gep['gene_idx']))
all_gep = pd.DataFrame(list(product(peco_idx, gene_idx)), columns = ['peco_idx', 'gene_idx'])
all_gep

,peco_idx,gene_idx
0,0,0
1,0,1
2,0,2
3,0,5
4,0,6
...,...,...
268243,31,12182
268244,31,12183
268245,31,12184
268246,31,12185


In [35]:
negative_gep = pd.concat([all_gep, positive_gep], axis = 0).drop_duplicates(keep = False)

In [36]:
all_gep.shape, positive_gep.shape, negative_gep.shape

((268248, 2), (23558, 2), (244690, 2))